In [1]:
import numpy as np
import scipy as sp
from geopy.distance import geodesic

def construct_C(loc_dict):
    """
    Computes Cost martrix from dictionary where 
    the keys are integers in the order of the arrays
    and the values are (lat,long) coordinates
    """
    key_order = np.sort(list(loc_dict.keys()))
    C = np.zeros((len(key_order),len(key_order)))
    for i, val_i in enumerate(key_order):
        for j, val_j in enumerate(key_order):
            C[i,j] = geodesic(val_i,val_j)
    return C
     
def wasserstein_distance(arr_1, arr_2, C,p=1):
    """
    Computes Wasserstein distribution between two 
    constant mass arrays. 
    """
    M = len(arr_1)
    N = len(arr_2)
    c = np.power(C.transpose().flatten(),p)
    B = np.zeros((M+N,M*N))
    B[:M,:] = np.kron(np.ones(N),np.eye(M))
    B[M:,:] = np.kron(np.eye(N),np.ones(M))
    g = np.zeros(M+N)
    g[:M] = arr_1
    g[M:] = arr_2
    sol = sp.optimize.linprog(c=c,A_eq=B,b_eq=g)
    return np.power(sol.fun,1/p)
    

In [26]:
def earth_movers_distance(arr_1, arr_2, C, p=1):
    m,n = len(arr_1),len(arr_2)
    mass_vec = [sum(arr_1),sum(arr_2)]
    min_mass = min(mass_vec)
    indx  = mass_vec.index(min_mass)
    len_slack = m*indx+n*(1-indx)
    v1 = arr_1/min_mass
    v2 = arr_2/min_mass
    part_cost = np.power(C.flatten(),p)
    cost = np.concatenate((part_cost,np.zeros(len_slack)))
    B = np.zeros((m+n,m*n+len_slack))
    B[:m,:m*n] = np.kron(np.eye(m),np.ones(n))
    B[m:m+n,:m*n] = np.kron(np.ones(n),np.eye(m))
    if indx ==0:
        B[m:m+n, -len_slack:] = np.eye(len_slack)
    else:
        B[:m, -len_slack:] = np.eye(len_slack)
    g = np.concatenate((v1,v2))
    sol = sp.optimize.linprog(c=cost,A_eq=B,b_eq=g)
    return np.power(sol.fun,1/p)


def earth_movers_signed(arr_1, arr_2, C, p=1):
    """
    Implemenetation of Earth Mover's with respect to the 
    OT plan laid out in:

    Piccoli, Benedetto, Francesco Rossi, and Magali Tournus.
    "A Wasserstein norm for signed measures, with application to nonlocal 
      transport equation with source term."
    arXiv preprint arXiv:1910.05105 (2019).

    NOTE: ARRAYS MUST BE OF THE SAME LENGTH
    """
    m,n = len(arr_1),len(arr_2)
    if not(m==n):
        raise ValueError("Arrays must be of the same size in this case")
    #Some jank to get the positive and negative parts
    arr_1 = np.array(arr_1)
    arr_2 = np.array(arr_2)
    pos_1 = arr_1.copy()
    pos_1[pos_1<0] = 0
    arr_1[arr_1>0] = 0
    neg_1 = -arr_1
    pos_2 = arr_2.copy()
    pos_2[pos_2<0] = 0
    arr_2[arr_2>0] = 0
    neg_2 = -arr_2
    v1 = pos_1 + neg_2
    v2 = pos_2 + neg_1
    mass_vec = [sum(v1),sum(v2)]
    min_mass = min(mass_vec)
    indx  = mass_vec.index(min_mass)
    v1 = v1/min_mass
    v2 = v2/min_mass
    part_cost = np.power(C.flatten(),p)
    cost = np.concatenate((part_cost,np.zeros(m)))
    B = np.zeros((2*m,m*(m+1)))
    B[:m,:m**2] = np.kron(np.eye(m),np.ones(n))
    B[m:2*m,:m**2] = np.kron(np.ones(n),np.eye(m))
    if indx ==0:
        B[m:2*m, -m:] = np.eye(m)
    else:
        B[:m,-m:] = np.eye(m)
    g = np.concatenate((v1,v2))
    sol = sp.optimize.linprog(c=cost,A_eq=B,b_eq=g)
    return np.power(sol.fun,1/p)





In [27]:
N = 100
tests_to_run = 10
passed = True
ED_arr = np.zeros(tests_to_run)
WD_arr = np.zeros(tests_to_run)
for n in range(tests_to_run):
    arr_1 = np.random.rand(N)
    arr_2 = np.random.rand(N)
    C = np.random.rand(N,N)
    p = np.random.randint(1,100)
    arr_1 = arr_1/sum(arr_1)
    arr_2 = arr_2/sum(arr_2)
    WD = wasserstein_distance(arr_1 = arr_1, arr_2 = arr_2,
                              C = C, p=p)
    ED = earth_movers_distance(arr_1 = arr_1, arr_2 = arr_2,
                               C = C, p =p)
    ED_arr[n] = ED
    WD_arr[n] = WD

In [2]:
N = 700
arr_1 = np.random.randn(N)
arr_2 = np.random.randn(N)
C = np.random.rand(N,N)
p = np.random.randint(1,100)
earth_movers_signed(arr_1 = arr_1, arr_2 = arr_2,
                    C = C, p = p)

NameError: name 'earth_movers_signed' is not defined

In [4]:
def generalized_signed_Wasserstein(arr_1,arr_2,C,a=1,b=1):
    """
    Implemenetation of 1 Generalized Wasserestin as laid out in:

    Piccoli, Benedetto, Francesco Rossi, and Magali Tournus.
    "A Wasserstein norm for signed measures, with application to nonlocal 
      transport equation with source term."
    arXiv preprint arXiv:1910.05105 (2019).

    NOTE: ARRAYS MUST BE OF THE SAME LENGTH
    """
    m,n = len(arr_1),len(arr_2)
    if not(m==n):
        raise ValueError("Arrays must be of the same size in this case")
    #Some jank to get the positive and negative parts
    arr_1 = np.array(arr_1)
    arr_2 = np.array(arr_2)
    pos_1 = arr_1.copy()
    pos_1[pos_1<0] = 0
    arr_1[arr_1>0] = 0
    neg_1 = -arr_1
    pos_2 = arr_2.copy()
    pos_2[pos_2<0] = 0
    arr_2[arr_2>0] = 0
    neg_2 = -arr_2
    v1 = pos_1 + neg_2
    v2 = pos_2 + neg_1
    part_cost = b*C.flatten()
    cost = np.concatenate((part_cost,a*np.ones(4*m)))
    B = np.zeros((2*m+1,m*(m+4)))
    #First Constraint
    B[:m,:m**2] = np.kron(np.eye(m),np.ones(m))
    B[:m,m**2:m*(1+m)] = np.eye(m)
    B[:m,m*(m+1):m*(m+2)] = -np.eye(m)
    #Second Constraint
    B[m:2*m,:m**2] = np.kron(np.ones(m),np.eye(m))
    B[m:2*m,m*(m+2):m*(m+3)] = np.eye(m)
    B[m:2*m,m*(m+3):m*(m+4)] = -np.eye(m)
    #Third Constraint
    B[-1,m*(m):m*(m+1)] = -np.ones(m)
    B[-1,m*(m+1):m*(m+2)] = np.ones(m)
    B[-1,m*(m+2):m*(m+3)] = np.ones(m)
    B[-1,m*(m+3):m*(m+4)] = -np.ones(m)

    g = np.concatenate((v1,v2,[sum(v2)-sum(v1)]))

    sol = sp.optimize.linprog(c=cost,A_eq=B,b_eq=g)
    return sol.fun

In [5]:
N = 700
a = np.random.randn(N)*100
b = np.random.randn(N)*10
C = np.random.rand(N,N)
generalized_signed_Wasserstein(a,b,C)

1104.2024827562461